In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Fetal-Health-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
  root_dir: Path
  model_path: Path
  test_path: Path
  metrics_file_path: Path

In [3]:
from FetalHealthC.constants import *
from FetalHealthC.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_path= config.test_path,
            metrics_file_path = config.metrics_file_path
        )
        
        return model_evaluation_config

In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
from FetalHealthC.utils.common import load_object 
from FetalHealthC.logging import logger
import yaml
 
class ModelEvaluation:
    def __init__(self, model_evaluation_config: ModelEvaluationConfig):
        self.config = model_evaluation_config

    def initiate_model_evaluation(self):
        # Fetching the test dataset
        data = pd.read_csv(self.config.test_path)
        logger.info("Test Dataset has been loaded successfully")

        # Spliting the data into x, y
        LABEL_COLUMN = 'fetal_health'
        x = data.drop(columns=LABEL_COLUMN)
        y = data[LABEL_COLUMN]
        logger.info("Data has been splitted successfully")

        # Fetching the model
        model = load_object(self.config.model_path)
        logger.info("Model has been loaded successfully")

        # Evaluating the model
        y_pred = model.predict(x)
        accuracy = accuracy_score(y, y_pred)
        cm = confusion_matrix(y, y_pred)
        logger.info(f"Accuracy score for model {accuracy}")
        logger.info(f"Confusion matrix for model {cm}")
        logger.info("Model evaluation has been completed successfully")

        # Writing the accuracy in the metrics yaml file
        content = dict()
        content['accuracy'] = str(accuracy)
        content['confusion_matrix'] = str(cm)

        with open(self.config.metrics_file_path, 'w') as f:
            yaml.dump(content,f)
        logger.info(f"{self.config.metrics_file_path} file is saved.")        

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.initiate_model_evaluation()

except Exception as e:
    raise e

[2024-01-13 17:04:13,819: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-13 17:04:13,821: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-13 17:04:13,822: INFO: common: created directory at: artifacts]
[2024-01-13 17:04:13,824: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-13 17:04:13,843: INFO: 1707977176: Test Dataset has been loaded successfully]
[2024-01-13 17:04:13,845: INFO: 1707977176: Data has been splitted successfully]
[2024-01-13 17:04:13,848: INFO: 1707977176: Model has been loaded successfully]
[2024-01-13 17:04:13,872: INFO: 1707977176: Accuracy score for model 0.9342723004694836]
[2024-01-13 17:04:13,873: INFO: 1707977176: Confusion matrix for model [[317  14   2]
 [  7  55   2]
 [  2   1  26]]]
[2024-01-13 17:04:13,874: INFO: 1707977176: Model evaluation has been completed successfully]
[2024-01-13 17:04:13,879: INFO: 1707977176: metrics.yaml file is saved.]
